In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Enter NCRB Individual PDF url
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-3.1_2019_0.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
# Download report
fileName = wget.download(pdf_url,fileName)

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-3.1_2019_0.pdf'

In [5]:
tables = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "48.096930533117934,736.8267655351573,547.3430694668822,52.29867117593344"
                              ],
                          columns=[
                              "76.9550888529887,175.0728271405493,210.66455573505655,238.56077544426498,271.26668820678515,297.2390306946688,328.02106623586434,354.95534733441036,386.6993214862682,418.4432956381261,447.30145395799684,482.8931825525041,519.4468497576737"
                              ],
                          split_text=True,
                          pages="all"
                         )

In [6]:
tables

<TableList n=2>

In [7]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [8]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [9]:
# Removes Garbage Rows that may have been detected Before the 'STATES' row
# AND
# Removes Garbage Rows that may have been detected after the 'TOTAL (ALL INDIA)' row
def clean(table):
    while(table.df.iloc[0,1] != 'STATES'):
        table.df = table.df.iloc[1:]
    while(table.df.iloc[-1,1] != 'TOTAL (ALL INDIA)'):
        table.df = table.df.iloc[:-1]
    # The Return line is unnecessary due to all changes being reflected in the original references
    return table

In [10]:
for each_table in tables:
    clean(each_table)

In [11]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
printnAllTables(tables)

Page No:  1
    0                  1  2  3  4  5   6  7  8   9  10 11 12 13
5                  STATES                                      
6    1     ANDHRA PRADESH  0  0  0  0   0  0  0   0  0  0  0  0
7    2  ARUNACHAL PRADESH  0  0  0  0   0  0  0   0  0  0  0  0
8    3              ASSAM  0  0  0  0   0  0  0   0  0  0  0  0
9    4              BIHAR  0  0  0  0   0  0  0   0  0  0  0  0
10   5       CHHATTISGARH  0  0  0  0   8  0  0   8  0  0  0  0
11   6                GOA  0  0  0  0   0  0  0   0  0  0  0  0
12   7            GUJARAT  0  0  0  0   0  0  0   0  0  0  0  0
13   8            HARYANA  0  0  0  0   0  0  0   0  0  0  0  0
14   9   HIMACHAL PRADESH  0  0  0  0   0  0  0   0  0  0  0  0
15  10    JAMMU & KASHMIR  1  0  0  1   6  0  0   6  0  0  0  0
16  11          JHARKHAND  0  0  0  0   0  0  0   0  0  0  0  0
17  12          KARNATAKA  0  0  0  0   0  0  0   0  0  0  0  0
18  13             KERALA  0  0  0  0   0  0  0   0  0  0  0  0
19  14     MADHYA PRADESH  0

In [12]:
# List of Column headers as column are nested, i.e exhibit a multi level/index structure
# The multi level structre will be generated and applied in the next 
column_header = [
    ['Deaths due to Forces of Nature','Killed in Action/Operation/Encounters/etc.','Fratricide'],
    ['Deaths due to Road/Railway Accidents','Deaths Due to Other Causes','Total']
]
len(column_header)

2

In [13]:
def set_Column(df, title):
    column_list = [
        ("Sl. No.",''),
        ("State/UT",''), 
        (title[0],'Male'),
        (title[0],'Female'),
        (title[0],'Transgender'),
        (title[0],'Total'),
        (title[1],'Male'),
        (title[1],'Female'),
        (title[1],'Transgender'),
        (title[1],'Total'),
        (title[2],'Male'),
        (title[2],'Female'),
        (title[2],'Transgender'),
        (title[2],'Total'),
    ]
    df.columns = pd.MultiIndex.from_tuples(column_list)

In [14]:
for i in range(len(tables)):
    set_Column(tables[i].df, column_header[i])
    tables[i].df.set_index(["Sl. No.","State/UT"],inplace=True)

In [15]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

# OR Print a simgle one according to your choice
# tables[23].df

In [16]:
# Convert List of tables to list of DataFrames
DataFrames = [each.df.copy() for each in tables]
# Concat all the tables one after the other, as they all have the same indexes.
final = pd.concat(DataFrames,axis=1)

In [17]:
# Reset Index before saving file, for better formatting in RAW CSV
final.reset_index().to_csv("Other Causes-wise Number of Cases, Persons Injured & Persons Died during 2019 (State & UT-wise).csv",index=False)